In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from bs4 import Comment
from functools import reduce
import os
from tqdm import tqdm
import argparse
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_all_player_links(url):
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    player_table_links = soup.find_all("a", text="View Player Stats")
    return player_table_links

def get_cl_links(url):
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    inner_nav_content = soup.select("#inner_nav")[0].contents
    uls = inner_nav_content[1].find_all("ul")
    links = []
    for i in range(len(uls[0].find_all("a"))):
        links.append(uls[0].find_all("a")[i].attrs['href'])
    return links

In [3]:
league = 'liga'

url = 'https://fbref.com/en/comps/9/Premier-League-Stats' if league == "premier" else " "
url = 'https://fbref.com/en/comps/12/La-Liga-Stats' if league == "liga" else " "
# url = 'https://fbref.com/en/comps/11/1476/2015-2016-Serie-A-Stats' if league == "serie" else " "
# url = 'https://fbref.com/en/comps/20/Bundesliga-Stats' if league == "bundes" else " "
# url = 'https://fbref.com/en/comps/13/Ligue-1-Stats' if league == "ligue" else " "
# url = "https://fbref.com/en/comps/24/Serie-A-Stats"
clurl = "https://fbref.com/en/comps/8/11323/stats/2021-2022-Champions-League-Stats"
player_table_links = get_all_player_links(url)
# clLinks = get_cl_links(clurl)
# print(player_table_links,end="\n")

In [4]:
season = input('Season?')
for link in player_table_links[::2]:
    r = requests.get("https://fbref.com" + link.attrs["href"])
    s = BeautifulSoup(r.content, "html.parser")
    comments = s.find_all(string=lambda text: isinstance(text, Comment))
    comments = sorted(comments, key=len, reverse=True)
    if comments is None:
        continue
    table = BeautifulSoup(comments[0], "lxml").find("table")
    df = pd.read_html(table.prettify(), flavor='bs4')[0]
    
    df.columns = [f"{y}_{x}" for x, y in df.columns.to_flat_index()]
    if "90s_" in df.columns:
        df["90s_"] = df["90s_"].fillna(0)

    df.columns = df.columns.str.replace("Unnamed: [0-9]+_level_0", "") 
    df = df.drop(columns = ["Rk_", "Matches_"])
    df = df.query("Player_ != 'Player'")
    df = df.drop_duplicates(subset=["Player_"], keep=False).reset_index(drop=True)
    for column in df.columns:
        if column[-1] == '_':
            df.rename(columns={f'{column}':f'{column[:-1]}'},inplace=True)
            
    skill = link.attrs['href'].split('#')[-1]
    path = f'/work/assets/fbref/leagueStats/{league}/{season}'
    if not os.path.exists(path):
        os.makedirs(path)
    
    df.to_csv(path + f'/{skill[10:]}.csv')

Season? 2223


In [6]:
for link in clLinks:
    r = requests.get("https://fbref.com" + link)
    s = BeautifulSoup(r.content, "html.parser")
    comments = s.find_all(string=lambda text: isinstance(text, Comment))
    comments = sorted(comments, key=len, reverse=True)
    if comments is None:
        continue
    table = BeautifulSoup(comments[0], "lxml").find("table")
    df = pd.read_html(table.prettify(), flavor='bs4')[0]
    df.columns = [f"{y}_{x}" for x, y in df.columns.to_flat_index()]
    if "90s_" in df.columns:
        df["90s_"] = df["90s_"].fillna(0)

    df.columns = df.columns.str.replace("Unnamed: [0-9]+_level_0", "") 
    df = df.drop(columns = ["Rk_", "Matches_"])
    df = df.query("Player_ != 'Player'")
    df = df.drop_duplicates(subset=["Player_"], keep=False).reset_index(drop=True)
    for column in df.columns:
        if column[-1] == '_':
            df.rename(columns={f'{column}':f'{column[:-1]}'},inplace=True)
            
    skill = link.split("/")[5]
    path = f'/work/assets/fbref/leagueStats/cl/2122/'
    if not os.path.exists(path):
        os.makedirs(path)
    df.to_csv(path+f'{skill}.csv')

In [5]:
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
import csv
import re
import sys
import getopt
import urllib
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

import time
import json
from tqdm import trange

class FbrefScrayper(object):

    def __init__(self, base_url='https://fbref.com/en/'):
        self.base_url = base_url    

        
    def build_df(self, df):

            columns = []
            try:
                for col1, col2 in df.columns.to_flat_index():
                    column = f'{col2}_{col1}'
                    columns.append(column)
            except:
                pass
            try:
                df.columns = columns
            except:
                pass
            if '90s_' in df.columns:
                df['90s_'] = df['90s_'].fillna(0)
                
            df.columns = df.columns.str.replace('Unnamed: [0-9]+_level_0', '')
            
            if 'Player_' in df.columns:
                try:
                    df = df[df['Player_'] != 'Player']
                except:
                    pass
                df = df.drop_duplicates(
                    subset=['Player_'],
                    keep=False).reset_index(drop=True)

            for column in df.columns:
                if column[-1] == '_':
                    df.rename(
                        columns={
                            f'{column}': f'{column[:-1]}'},
                        inplace=True)
            if 'Matches' in df.columns:
                df = df.drop(columns='Matches')

            return df
        
    
    def get_team_data(self, url=None, number=0):
        """
        number->0: Standard Stats
                1: Score & Fixtures
                2: GoalKeeping
                3: Advanced Goalkeeping
                4: Shooting
                5: Passing
                6: Pass Types
                7: Goal and Shot Creation
                8: Defensive Actions
                9: Possession
                10: Playing Time
                11: Miscellaneous Stats
                12: Regular Season 
        """
        if url is None:
            url = 'squads/206d90db/Barcelona-Stats'
        html = requests.get(self.base_url + url)
        soup = BeautifulSoup(html.content, 'html.parser')
        table = soup.find_all('table')
        df = pd.read_html(table[number].prettify(), flavor='bs4')[0]
        return self.build_df(df)
    
    def get_match_data(self, url=None, number=0):
        """
        number->0: Member Home
                1: Member Away
                2: Match Stats
                3: Summary Home
                4: Passing Home
                5: Pass Types Home
                6: Defensive Actions Home
                7: Possession Home
                8: Miscellaneous Stats Home
                9: Goalkeeper Stats Home
                10: Summary Away
                11: Passing Away
                12: Pass Types Away
                13: Defensive Actions Away 
                14: Possessiong Away
                15: Miscellaneous Stats Away
                16: Goalkeeper Stats Away
                17: Shots Data
        """
        if url is None:
            url = 'https://fbref.com/en/matches/086be9c6/Valladolid-Barcelona-December-22-2020-La-Liga'

        html = requests.get(url)
        comment = re.compile('<!--|-->')
        soup = BeautifulSoup(comment.sub("", html.text), 'lxml')
        table = soup.find_all("table")
        df = pd.read_html(table[number].prettify(), flavor='bs4')[0]
        return self.build_df(df)

    def get_player_season_data(self, url='None',number=4):
        """
        number->0: Scouting Report 
                1: Scouting Report 
                2: Standard Stat
                3: Shooting
                4: Passing
                5: Pass Types
                6: Goal and Shot Creation
                7: Defensive Actions
                8: Possession
                9: Playing Time
                10: Miscellaneous Stats
        """

        if url is None:
            url = self.base_url + 'players/d70ce98e/Lionel-Messi'

        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser')
        table = soup.find_all('table')
        df = pd.read_html(table[number].prettify(), flavor='bs4')[0]
        return self.build_df(df)


    def get_player_match_data(self, url=None,category='summary'):
        """
        category->summary
                  passing
                  passing_types
                  gca
                  defense
                  possession
                  misc
        """
        if url is None:
            url = self.base_url + f'players/d70ce98e/matchlogs/2020-2021/{category}/Lionel-Messi-Match-Logs'

        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser')
#         comment = re.compile('<!--|-->')
#         soup = BeautifulSoup(comment.sub("", html.text), 'lxml')
        table = soup.find_all('table')
        df = pd.read_html(table[0].prettify(), flavor='bs4')[0]
        return self.build_df(df)


In [4]:
scrayper = FbrefScrayper()

## Team Stats

In [13]:
# target_url = 'squads/db3b9613/2020-2021/Atletico-Madrid-Stats'
# target_url = "squads/206d90db/2020-2021/Barcelona-Stats"
# target_url = "squads/53a2f082/2020-2021/Real-Madrid-Stats"

df1 = scrayper.get_team_data(url=target_url,number=0)
df2 = scrayper.get_team_data(url=target_url,number=4)
df3 = scrayper.get_team_data(url=target_url,number=5)
df4 = scrayper.get_team_data(url=target_url,number=6)
df5 = scrayper.get_team_data(url=target_url,number=7)
df6 = scrayper.get_team_data(url=target_url,number=8)
df7 = scrayper.get_team_data(url=target_url,number=9)
df8 = scrayper.get_team_data(url=target_url,number=11)

In [14]:
season = 2021
fixture = scrayper.get_team_data(url=target_url,number=1)
fixture.to_csv(f"/work/assets/fbref/teamStats/real-madrid/fixture{season}.csv")

In [11]:
df = df1.merge(df2,on='Player',how='inner')
df = df.merge(df3,on='Player',how='inner')
df = df.merge(df4,on='Player',how='inner')
df = df.merge(df5,on='Player',how='inner')
df = df.merge(df6,on='Player',how='inner')
df = df.merge(df7,on='Player',how='inner')
df = df.merge(df8,on='Player',how='inner')

target = "barcelona"
path = f'/work/assets/fbref/teamStats/{target}/team/'
import os
if not os.path.exists(path):
    os.makedirs(path)

df.to_csv(path + f'{season}.csv')

In [18]:
cate = {0: 'MemberHome',
        1: 'MemberAway',
        2: 'MatchStats',
        3: 'SummaryHome',
        4: 'PassingHome',
        5: 'PassTypesHome',
        6: 'DefensiveHome',
        7: 'PossessionHome',
        8: 'MiscellaneousHome',
        9: 'GoalkeeperHome',
        10: 'SummaryAway',
        11: 'PassingAway',
        12: 'PassTypesAway',
        13: 'DefensiveAway', 
        14: 'PossessiongAway',
        15: 'MiscellaneousAway',
        16: 'GoalkeeperAway',
        17: 'Shots'}

for i in range(39):
    df = scrayper.get_match_data(url='https://fbref.com/en/matches/c2818e17/Barcelona-Getafe-August-29-2021-La-Liga',
                                 number=i)
    path = f'/work/assets/fbref/teamStats/barcelona/match/{season}/#3/'
    if not os.path.exists(path):
        os.makedirs(path)
    df.to_csv(path + f'{cate[i]}.csv')

KeyError: 18

## Player Match logs

In [5]:
playerName = "Marc-Andre-ter-Stegen"
for category in ['summary', 'passing', 'passing_types', 'gca',
                 'defense', 'possession','misc']:
    turl = f'https://fbref.com/en/players/df69b544/matchlogs/2022-2023/{category}/{playerName}-Match-Logs'
    df = scrayper.get_player_match_data(url=turl)
    fileName = turl.split("/")[-2]
    path = f"/work/assets/fbref/playerStats/matchLogs/{playerName}/"
    if not os.path.exists(path):
        os.makedirs(path)
    df.to_csv(path + f"{fileName}.csv")

In [9]:
for season in ["2022-2023"]: #["2014-2015","2015-2016","2016-2017","2017-2018","2018-2019","2019-2020","2020-2021","2021-2022"]:
    playerName = "Marc-Andre-ter-Stegen"
    url = f'https://fbref.com/en/players/6f51e382/matchlogs/{season}/keeper/{playerName}-Match-Logs'
    print(url)
    df = scrayper.get_player_match_data(url=url)
    categoryName = url.split("/")[-2]
    fileName = url.split("/")[-3]
    path = f"/work/assets/fbref/playerStats/matchLogs/{playerName}/{categoryName}/"
    if not os.path.exists(path):
        os.makedirs(path)
    df.to_csv(path + f"{fileName}.csv")

https://fbref.com/en/players/6f51e382/matchlogs/2022-2023/keeper/Marc-Andre-ter-Stegen-Match-Logs


## team match logs

In [22]:
season = "2122"

url = 'https://fbref.com/en/squads/db3b9613/Atletico-Madrid-Stats'
html = requests.get(url)
soup = BeautifulSoup(html.content,'html.parser')
table = soup.find_all('table')
tbody = table[0].find('tbody')
# table = BeautifulSoup(table[0], "lxml").find("table")
df = pd.read_html(table[1].prettify(), flavor='bs4')[0]
fileName = url.split("/")[-1][:-6].lower()

path = f'/work/assets/fbref/teamStats/{fileName}/match/{season}/'
if not os.path.exists(path):
    os.makedirs(path)
df.to_csv(path + f'fixtures.csv')

## Player Season Stats

In [22]:
category = ['summary', 'shot', 'passing', 'passing_types', 'gca', 'defense', 'possession','playtime','misc']
for i in range(len(category)):
    _i = i + 2
    url = 'https://fbref.com/en/players/08f5afaa/Gianluigi-Donnarumma'
    df = scrayper.get_player_season_data(url,number=_i)
    fileName = url.split("/")[-1]
    df.to_csv(f"/work/assets/fbref/seasonStats/{fileName}-{category[i]}.csv")
    

In [17]:
df = pd.read_csv('/work/assets/fbref/teamStats/atletico-madrid/match/2122/fixtures.csv')

df = df.dropna(subset=['xG'])

df['xgSMA'] = df['xG'].rolling(window=3).mean()
df['xgaSMA'] = df['xGA'].rolling(window=3).mean()

df = df.reset_index(drop=True)

# import matplotlib.pyplot as plt

# plt.plot(df.index,df.xgSMA,color='blue')
# plt.plot(df.index,df.xgaSMA,color='red')

## League Table

In [12]:
# url = "https://fbref.com/en/comps/12/La-Liga-Stats"
url = "https://fbref.com/en/comps/9/Premier-League-Stats"
html = requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')
table = soup.find_all('table')

In [13]:
df = pd.read_html(table[0].prettify(), flavor='bs4')[0]

In [14]:
# team id
table[0].find_all("a", href=re.compile("/en/squads/"))[0]["href"].split("/")[3]
# team name
table[0].find_all("a", href=re.compile("/en/squads/"))[0]["href"].split("/")[4].split("-")[0]

'Arsenal'

In [15]:
team_ids = []
for i in range(len(table[0].find_all("a", href=re.compile("/en/squads/")))):
    team_id = table[0].find_all("a", href=re.compile("/en/squads/"))[i]["href"].split("/")[3]
    team_ids.append(team_id)
    
team_names = []
for i in range(len(table[0].find_all("a", href=re.compile("/en/squads/")))):
    if len(table[0].find_all("a", href=re.compile("/en/squads/"))[i]["href"].split("/")[4].split("-")) == 3:
        team_name = table[0].find_all("a", href=re.compile("/en/squads/"))[i]["href"].split("/")[4].split("-")[0] + "-" + \
                    table[0].find_all("a", href=re.compile("/en/squads/"))[i]["href"].split("/")[4].split("-")[1]
    else:    
        team_name = table[0].find_all("a", href=re.compile("/en/squads/"))[i]["href"].split("/")[4].split("-")[0]
    team_names.append(team_name.lower())

In [16]:
print(team_ids)
print(team_names)

['18bb7c10', 'b8fd03ef', 'b2b47a98', '19538871', '361ca564', '822bd0ba', 'fd962109', 'd07537b9', 'cd051869', 'cff3d9bb', '8602292d', '47c64c55', 'a2d435b3', '5bfb9659', 'e4a775cb', '4ba7cbea', '7c21e445', 'd3fd31cc', '8cec06e1', '33c895d4']
['arsenal', 'manchester-city', 'newcastle-united', 'manchester-united', 'tottenham-hotspur', 'liverpool', 'fulham', 'brighton', 'brentford', 'chelsea', 'aston-villa', 'crystal-palace', 'leicester-city', 'leeds-united', 'nottingham-forest', 'bournemouth', 'west', 'everton', 'wolverhampton-wanderers', 'southampton']


In [17]:
seasons = ["2020-2021","2021-2022","2022-2023"]
def get_fixtures(team_id="206d90db",team_name=None):
    for i,season in enumerate(seasons):
        if i == len(seasons)-1:
            target_url = f"squads/{team_id}/{team_name.title()}-Stats"
        else:    
            target_url = f"squads/{team_id}/{season}/{team_name.title()}-Stats"
            
        fixture = scrayper.get_team_data(url=target_url,number=1)
        save_to_path = f"/work/assets/fbref/teamStats/{team_name}/"
        if not os.path.exists(save_to_path):
            os.makedirs(save_to_path)
        fixture.to_csv(save_to_path + f"fixture{season}.csv")
for team_id, team_name in zip(team_ids, team_names):
    get_fixtures(team_id=team_id,team_name=team_name)

IndexError: list index out of range

In [6]:
import requests
from bs4 import BeautifulSoup

In [7]:
html = requests.get("https://fbref.com/en/squads/206d90db/Barcelona-Stats")
soup = BeautifulSoup(html.content, 'html.parser')
table = soup.find_all('table')[0]
table.find_all("th", csk=True)[0].get("data-append-csv")

<table class="stats_table sortable min_width" data-cols-to-freeze=",1" id="stats_standard_12"> <caption>Standard Stats <span style="color: #666; font-size:smaller">2022-2023 Barcelona: La Liga</span> Table</caption> <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup> <thead> <tr class="over_header"> <th aria-label="" class="over_header center" colspan="4" data-stat=""></th><th></th> <th aria-label="" class="over_header center group_start" colspan="3" data-stat="header_playing">Playing Time</th> <th aria-label="" class="over_header center group_start" colspan="7" data-stat="header_performance">Performance</th> <th aria-label="" class="over_header center group_start" colspan="5" data-stat="header_per90">Per 90 Minutes</th> <th aria-label="" class="over_header center group_start" colspan="4" data-stat="header_expected">Expected</th> <th aria

'6f51e382'

In [62]:
league = "liga"
season = 2223
df1 = pd.read_csv(f"/work/assets/fbref/leagueStats/{league}/{season}/standard.csv")
df2 = pd.read_csv(f"/work/assets/fbref/leagueStats/{league}/{season}/shooting.csv")
df3 = pd.read_csv(f"/work/assets/fbref/leagueStats/{league}/{season}/passing.csv")
df4 = pd.read_csv(f"/work/assets/fbref/leagueStats/{league}/{season}/passing_types.csv")
df5 = pd.read_csv(f"/work/assets/fbref/leagueStats/{league}/{season}/gca.csv")
df6 = pd.read_csv(f"/work/assets/fbref/leagueStats/{league}/{season}/defense.csv")
df7 = pd.read_csv(f"/work/assets/fbref/leagueStats/{league}/{season}/possession.csv")
df8 = pd.read_csv(f"/work/assets/fbref/leagueStats/{league}/{season}/playing_time.csv")
df9 = pd.read_csv(f"/work/assets/fbref/leagueStats/{league}/{season}/misc.csv")

df = df1.merge(df2, how="inner", on=["Player", "Nation"], suffixes=["", "_duplicated"]) \
        .merge(df3, how="inner", on=["Player", "Nation"], suffixes=["", "_duplicated"]) \
        .merge(df4, how="inner", on=["Player", "Nation"], suffixes=["", "_duplicated"]) \
        .merge(df5, how="inner", on=["Player", "Nation"], suffixes=["", "_duplicated"]) \
        .merge(df6, how="inner", on=["Player", "Nation"], suffixes=["", "_duplicated"]) \
        .merge(df7, how="inner", on=["Player", "Nation"], suffixes=["", "_duplicated"]) \
        .merge(df8, how="inner", on=["Player", "Nation"], suffixes=["", "_duplicated"]) \
        .merge(df9, how="inner", on=["Player", "Nation"], suffixes=["", "_duplicated"])

not_duplicated_cols = [col for col in df.columns if not "_duplicated" in col]
df = df[not_duplicated_cols]

df.rename(columns=rename_dict, inplace=True)
df.reset_index(inplace=True, drop=True)
df = df.loc[:,~df.columns.duplicated()]

In [63]:
not_90_min_cols = [col for col in df.columns if not "90" in col and not "Per90" in col and not "%" in col and not "Goal-xG(non-pen)" in col and not "xG Plus/Minus" in col]

In [67]:
df.loc[:,not_90_min_cols[11]:] = df.loc[:,not_90_min_cols[11]:].apply(lambda x: round(x / df["90s"],2))

In [76]:
df[df["Player"].apply(str).str.contains("Roberto")]

,Unnamed: 0,Player,Nation,Pos,Squad,Age,Born,MP Playing Time,Starts,Min Playing Time,...,Crs_Performance,Intercept,Tackle\nWin,PKwon_Performance,PKcon_Performance,OG_Performance,Recovery,Won\nAerial Duels,Lost\nAerial Duels,Won%\nAerial Duels
33,33,Roberto Arroyo,es ESP,MF,Valladolid,19-206,2003,1,0,15,...,5.00,0.00,5.00,0.0,0.0,0.0,0.00,0.00,0.00,NaN
207,207,Roberto González,es ESP,"FW,MF",Betis,22-072,2001,2,0,54,...,8.33,1.67,0.00,0.0,0.0,0.0,0.00,0.00,0.00,NaN
370,373,Roberto Navarro,es ESP,"MF,FW",Real Sociedad,20-341,2002,16,1,342,...,3.95,0.53,0.79,0.0,0.0,0.0,5.79,1.58,1.32,14.34
442,445,Sergi Roberto,es ESP,"DF,MF",Barcelona,31-040,1992,16,9,871,...,1.75,0.82,1.34,0.0,0.0,0.0,4.74,0.31,0.93,2.58
443,446,Lucas Robertone,ar ARG,MF,Almería,26-001,1997,25,24,1922,...,4.95,1.45,0.98,0.0,0.0,0.0,6.54,0.75,1.40,1.63
503,506,Roberto Torres,es ESP,"FW,MF",Osasuna,34-012,1989,2,0,11,...,20.00,0.00,0.00,0.0,0.0,0.0,10.00,0.00,0.00,NaN
